In [2]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0
!pip3 install -q -U Cython
!pip3 install -q -U wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 8.1 MB/s eta 0:00:00
     ━━━━

In [1]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

!wandb login
import wandb

%env WANDB_PROJECT=DollyTrain-Gemma7bit-webnlg

import torch
import transformers
import bitsandbytes as bnb
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

wandb: Currently logged in as: rpucs. Use `wandb login --relogin` to force relogin
env: WANDB_PROJECT=DollyTrain-Gemma7bit-webnlg


In [2]:
model_ids = [
    ("google/gemma-2b-it", "gemma"),
    ("google/gemma-7b-it", "gemma")
    #("mistralai/Mistral-7B-Instruct-v0.2", "mistral")
]
def formatting_prompts_func(dataset):
    output_texts = []
    for datapoint in dataset:
        text = f"### User input: {datapoint['instruction']} \n### Relations: {' '.join(['<entry><head>{}<rel>{}<tail>{}'.format(rel['head'], rel['type'], rel['tail']) for rel in datapoint['context']]) } \n### Response:{datapoint['response']}<eos>"
        output_texts.append(text)
    return output_texts

def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [3]:
train = load_dataset("rPucs/TripletDollyQA-3k-Gemma-Nlg", "train")["train"]
test = load_dataset("rPucs/TripletDollyQA-3k-Gemma-Nlg", "test")["train"]


prompts = formatting_prompts_func(train)
train = train.add_column("prompt", prompts)
prompts = formatting_prompts_func(test)
test = test.add_column("prompt", prompts)

In [4]:
print(len(train))
print([test["prompt"][0]])

2400
['### User input: What is the Fallout game? \n### Relations: <entry><head>Fallout:_A_Post-Nuclear_Role-Playing_Game<rel>prehistoricdeception<tail>"Vault Dweller infiltrates Vault 13, looking for supplies" <entry><head>Fallout:_A_Post-Nuclear_Role-Playing_Game<rel>prehistoricdeception<tail>World_War_III <entry><head>Fallout:_A_Post-Nuclear_Role-Playing_Game<rel>publisher<tail>Interplay_Productions <entry><head>Fallout:_A_Post-Nuclear_Role-Playing_Game<rel>released<tail>1997 <entry><head>Fallout:_A_Post-Nuclear_Role-Playing_Game<rel>setting<tail>"Underground shelter Vault 13, a haven for the Commonwealth of Nations" <entry><head>Fallout:_A_Post-Nuclear_Role-Playing_Game<rel>setting<tail>"Underground, a decade after World War III" <entry><head>Vault_13<rel>location<tail>Vault_13 <entry><head>Vault_13<rel>status<tail>"Incapacitated, but not deactivated" <entry><head>Vault_13<rel>status<tail>"Operated by the Commonwealth of Nations" <entry><head>World_War_III<rel>country<tail>China \n#

In [5]:
# https://github.com/huggingface/trl/issues/588
# https://huggingface.co/docs/trl/v0.7.2/en/sft_trainer#using-tokenids-directly-for-responsetemplate

#text = f"### User input: {datapoint['instruction']} \n### Relations: {' '.join(['<entry><head>{}<rel>{}<tail>{}'.format(rel['head'], rel['type'], rel['tail']) for rel in datapoint['context']]) } \n### Response: {datapoint['response']}<eos>"
text = """### Relations:
### Response:* Leisure, a 1976 animated fillm
* Art, an animated film
* Australian History, an animated film
* Hearts and Minds, an animated film
* Karl Marx, an animated film
* Man Environment Machine, a machine sculpture featured at World Expo '85
* Global Haywire, a 2007 documentary
* Petty's Parallel Worlds, a 2008 book<eos>"""
response_template = " \n### Response:"
first = tokenizer.encode(response_template, add_special_tokens=False)[0:]
second = tokenizer.encode(text, add_special_tokens=False)
print(first, second)

NameError: name 'tokenizer' is not defined

In [5]:
# save
# model_id = "google/gemma-7b-it"
# new_model="gemma-7b-itTripletDolly-WebNLG"
# base_model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     low_cpu_mem_usage=True,
#     return_dict=True,
#     torch_dtype=torch.float16,
#     device_map={"": 0},
# )
# tokenizer = AutoTokenizer.from_pretrained(model_id,
#                                           add_eos_token=True,
#                                           padding_side="right"
#                                           )
# merged_model= PeftModel.from_pretrained(base_model, new_model)
# merged_model= merged_model.merge_and_unload()
#
# # Save the merged model
# merged_model.save_pretrained("merged_model",safe_serialization=True)
# tokenizer.save_pretrained("merged_model")
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"
# # Push the model and tokenizer to the Hugging Face Model Hub
# merged_model.push_to_hub(new_model, use_temp_dir=False)
# tokenizer.push_to_hub(new_model, use_temp_dir=False)

In [6]:
# https://medium.com/@mayvic/solving-the-issue-of-falcon-text-generation-never-stopping-e8f599eae8f0
# PADDING PROBLEM - PADDING TOKEN = EOS TOKEN

In [4]:
# Setup train config
model_id = "google/gemma-7b-it"
new_model = model_id.split("/")[1] + "TripletDolly-WebNLG-fullcollator" #Name of the model you will be pushing to huggingface model hub
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side="right"

In [5]:


max_steps=200
logging_steps=1
run_name="run-1epoch-batch1-noSpaceAfterResponse:-fullCollator"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
#modules = find_all_linear_names(model)
#lora_config = LoraConfig(
#  r=64,
#  lora_alpha=32,
#  target_modules=modules,
#  lora_dropout=0.05,
#  bias="none",
#  task_type="CASUAL_LM"
#)
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

# setup tokens
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# with torch.no_grad():
#   model.resize_token_embeddings(len(tokenizer))
# model.config.pad_token_id = tokenizer.pad_token_id
# tokenizer.pad_token = "[PAD]"
tokenizer.padding_side="right"





trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

response_template_string = " \n### Response:"
response_template = tokenizer.encode(response_template_string, add_special_tokens=False)[1:-1]
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    dataset_text_field="prompt",
    #peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        #max_steps=max_steps,
        num_train_epochs=1,
        learning_rate=2e-4,
        logging_steps=logging_steps,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
        report_to="wandb",
        run_name=run_name
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    #data_collator=collator
    #max_seq_length=10_000,
    #packing=False
)

# Train
#model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()
#model.config.use_cache = True

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

...

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
wandb: Currently logged in as: rpucs. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,7.960000
2,8.465700
3,7.510400
4,9.413800
5,7.991700
6,6.213000
7,4.051500
8,3.931500
9,9.840200
10,3.010500


Checkpoint destination directory outputs/checkpoint-600 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=600, training_loss=1.445767834285895, metrics={'train_runtime': 6712.2038, 'train_samples_per_second': 0.358, 'train_steps_per_second': 0.089, 'total_flos': 2.722857356132352e+16, 'train_loss': 1.445767834285895, 'epoch': 1.0})

In [7]:
# Upload
print("Uploading {} to hub".format(new_model))
#trainer.model.save_pretrained(new_model)
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
 #tokenizer.pad_token = tokenizer.eos_token
 #tokenizer.padding_side = "right"
# Push the model and tokenizer to the Hugging Face Model Hub
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

# Release memory
del model, merged_model, tokenizer, base_model, trainer
torch.cuda.empty_cache()

Uploading gemma-7b-itTripletDolly-WebNLG-fullcollator to hub


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

NameError: name 'model' is not defined

<function posix._exit(status)>

# Test out model

In [ ]:
print(len(train))
print(len(test))

2400
600


In [5]:
def get_completion(prompt: str, model, tokenizer) -> str:

  device = "cuda:0"
  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
  model_inputs = encodeds.to(device)
  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True) #, pad_token_id="[PAD]")
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=False)
  return (decoded)

def get_completion_batch(prompt: list[str], model, tokenizer) -> list[str]:
  device = "cuda:0"
  encodeds = tokenizer(prompt,\
                      return_tensors="pt", add_special_tokens=False,\
                      padding=True)
  model_inputs = encodeds.to(device)
  generated_ids = model.generate(**model_inputs, max_new_tokens=600, do_sample=True) #,pad_token_id="[PAD]")
  decoded = tokenizer.batch_decode(generated_ids)
  return decoded



In [6]:
#config = PeftConfig.from_pretrained("rPucs/gemma-2b-relextract-nospecialtoken-webnlg")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quent_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

#new_model = model_id.split("/")[1] + "TripletDolly-WebNLG-fullcollator"
model = AutoModelForCausalLM.from_pretrained(new_model, quantization_config=bnb_config, device_map={"":0})

tokenizer = AutoTokenizer.from_pretrained(new_model,
                                          add_eos_token=True
                                          )

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
for idx in range(10):
  prompt = test[idx]["prompt"]
  f = prompt.find(" \n### Response:")
  input = prompt[0:f+len(" \n### Response:")]
  expected_output = prompt.split(" \n### Response:")[1]

  output = get_completion(input, model, tokenizer)
  output_mod = output[len(input):]
  print(idx)
  print("Input: {}".format(input))
  print("Expected output: {}".format(expected_output))
  print("\nModel output: {}".format(output_mod))

0
Input: ### User input: What is the Fallout game? 
### Relations: <entry><head>Fallout:_A_Post-Nuclear_Role-Playing_Game<rel>prehistoricdeception<tail>"Vault Dweller infiltrates Vault 13, looking for supplies" <entry><head>Fallout:_A_Post-Nuclear_Role-Playing_Game<rel>prehistoricdeception<tail>World_War_III <entry><head>Fallout:_A_Post-Nuclear_Role-Playing_Game<rel>publisher<tail>Interplay_Productions <entry><head>Fallout:_A_Post-Nuclear_Role-Playing_Game<rel>released<tail>1997 <entry><head>Fallout:_A_Post-Nuclear_Role-Playing_Game<rel>setting<tail>"Underground shelter Vault 13, a haven for the Commonwealth of Nations" <entry><head>Fallout:_A_Post-Nuclear_Role-Playing_Game<rel>setting<tail>"Underground, a decade after World War III" <entry><head>Vault_13<rel>location<tail>Vault_13 <entry><head>Vault_13<rel>status<tail>"Incapacitated, but not deactivated" <entry><head>Vault_13<rel>status<tail>"Operated by the Commonwealth of Nations" <entry><head>World_War_III<rel>country<tail>China 
#

In [ ]:
os._exit(0)